# 4. Assignment

## Portfolio Management
### MLP Training

#### January 2025

Questions?
* Contact: Mark Hendricks
* hendricks@uchicago.edu

***

# 1. Carry in Equities

### Data
You will need the files in the github repo, 
* `data/spx_carry_weekly.xlsx`
* `data/spx_returns_weekly.xlsx`

### 1. Data Processing
* Take the intersection of dates of the two data sets, so you have one unified set of dates.
* Report the maximum and minimum dividend yields from the entire panel of data. To which stocks did they belong?

In [111]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

SCALE = 52
dvd_yld_df = pd.read_excel('../data/spx_carry_weekly.xlsx', 's&p500 dvd yield').set_index('date')
ret_df = pd.read_excel('../data/spx_returns_weekly.xlsx', 's&p500 rets').set_index('date')

In [112]:
display(dvd_yld_df.head())
display(ret_df.head())

,A,AAPL,ABBV,ABT,ACN,ADI,ADM,ADP,AEE,AEP,...,WST,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.009862,0.017196,0.029747,0.021381,0.022963,0.026647,0.018454,0.023532,0.035660,0.034669,...,0.008436,0.026936,0.032124,0.040847,0.033223,0.029732,0.013445,0.031524,0.008057,0.007666
2015-01-09,0.009855,0.016784,0.029796,0.021239,0.022722,0.027047,0.019664,0.023179,0.036444,0.034393,...,0.008496,0.026816,0.031763,0.040255,0.033167,0.029967,0.014476,0.031048,0.007674,0.007503
2015-01-16,0.010458,0.017738,0.030369,0.021578,0.022950,0.027566,0.020134,0.023176,0.035239,0.033349,...,0.008484,0.027322,0.031668,0.040819,0.032415,0.030290,0.014853,0.031489,0.007753,0.007508
2015-01-23,0.010307,0.016640,0.031395,0.021888,0.022865,0.027151,0.020054,0.022568,0.035398,0.032858,...,0.008679,0.027759,0.032186,0.041093,0.031780,0.030366,0.014558,0.030776,0.007754,0.007525
2015-01-30,0.010590,0.016046,0.032477,0.021448,0.024277,0.028404,0.020588,0.023749,0.036219,0.033753,...,0.008923,0.027714,0.032357,0.040554,0.031974,0.031572,0.015015,0.031555,0.008092,0.007770


,A,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15,-0.038352,0.001776,0.041190,0.003094,-0.028354,0.010896,0.015026,-0.003202,-0.083744,-0.002688,...,-0.035071,-0.074553,0.010911,0.008844,0.038694,-0.040441,-0.008856,-0.004628,-0.045570,-0.054490
2016-01-22,0.023443,0.044184,0.025986,-0.012578,-0.017597,0.029919,0.005159,0.031313,0.054585,0.040940,...,-0.005154,-0.015493,0.013707,0.003561,-0.013019,0.031835,0.028171,-0.019094,-0.000884,0.002658
2016-01-29,-0.008688,-0.040268,-0.066803,-0.054461,0.034298,0.032277,-0.005578,0.048268,0.063800,0.024411,...,0.022510,0.007475,0.138075,0.043407,0.016717,0.056423,0.049144,0.001109,0.069027,-0.008065
2016-02-05,-0.042762,-0.028849,-0.032423,-0.011625,-0.023833,-0.062252,-0.111410,-0.075010,-0.036211,-0.021423,...,-0.003931,-0.063255,-0.080634,0.034274,0.028646,0.019748,-0.036065,-0.042816,-0.050000,-0.049711
2016-02-12,0.004993,-0.000327,-0.010165,-0.007484,0.037305,-0.043144,-0.031944,-0.003612,-0.038858,0.005288,...,-0.054202,-0.057943,0.116782,-0.017960,0.021046,0.010367,-0.034689,-0.033997,0.033635,-0.011732


In [113]:
dvd_yld_df, ret_df = dvd_yld_df.align(ret_df, join="inner", axis=0)

### 2. Static Carry
* Starting at the beginning of the sample, build a portfolio where you go (equally) long the top 100 stocks as ranked by dividend yield, and you short (equally) the smallest 100 stocks as ranked by dividend yield.
* Using those weights, calculate the return of your portfolio over time. 

Report the following annualized stats
* mean
* volatility
* Sharpe

In [114]:
positions_df = pd


In [115]:
filtered_ret_df, filtered_positions = ret_df.align(positions, join="inner", axis=1)

filtered_ret_df.shape, filtered_positions.shape

((468, 392), (392,))

# Statistics on portfolio

In [116]:
port_ret = filtered_ret_df.dot(filtered_positions)
port_ret.name = 'Return'

In [117]:
port_mean_ret = port_ret.mean() * SCALE
port_mean_ret

np.float64(-0.025910651178246823)

In [118]:
port_vol = port_ret.std() * np.sqrt(SCALE)
port_vol

np.float64(0.06135604930033632)

In [119]:
port_sharpe_ratio = port_mean_ret / port_vol
port_sharpe_ratio

np.float64(-0.4222998624213048)

In [120]:
print(pd.DataFrame(data=filtered_positions[filtered_positions < 0]).to_string())

      Static Positions
A                -0.01
ALLE             -0.01
AME              -0.01
AON              -0.01
AOS              -0.01
APH              -0.01
AVGO             -0.01
BAC              -0.01
BALL             -0.01
BAX              -0.01
BF/B             -0.01
BRO              -0.01
C                -0.01
CBOE             -0.01
CDW              -0.01
CI               -0.01
COO              -0.01
COR              -0.01
COST             -0.01
CTAS             -0.01
CTRA             -0.01
DAL              -0.01
DG               -0.01
DHI              -0.01
DHR              -0.01
DIS              -0.01
DPZ              -0.01
ECL              -0.01
EFX              -0.01
EL               -0.01
EOG              -0.01
EQT              -0.01
EXPE             -0.01
FDS              -0.01
FDX              -0.01
FICO             -0.01
GL               -0.01
GPN              -0.01
GS               -0.01
HII              -0.01
HRL              -0.01
HUM              -0.01
ICE        

### 3. Challenges

Mention two challenges of this strategy.

We are shorting Tech stocks that perform really well:
- NVIDIA
- GOOG/GOOGL
- META

We go long on energy and utilities. This is terrible considering the past years were we have explosive growth in the tech sector. We need someway to hedge out the tech factor in this strategy.

### 4. Dynamic Carry

Make the carry strategy of `2` dynamic by recalculating the weights every period.

Report the performance statistics.

t+1 (r t) (position t+1)
t   (r t-1) (position t)
...

We need

t+1 (r t) (position t+1)
t   (r t-1) (position t)

so shift positions... 

In [121]:
def adjust_positions(prev_dvd_yield: pd.Series):
    sorted_yld = prev_dvd_yield.sort_values()
    
    losers = sorted_yld[0:100].index
    winners = sorted_yld[-100:].index

    positions_df = pd.Series(0, index=prev_dvd_yield.index)
    positions_df[losers] = -0.01
    positions_df[winners] = 0.01

    return positions_df

In [128]:
positions_df = dvd_yld_df.apply(adjust_positions, axis=1)
shifted_positions_df = positions_df.shift(1)
filtered_ret_df, filtered_positions_df = ret_df.align(shifted_positions_df, join='inner', axis=1)

/var/folders/jv/chfp3b1d6yqbt4n29m5mxx040000gp/T/ipykernel_68069/2076384319.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  positions_df[losers] = -0.01
/var/folders/jv/chfp3b1d6yqbt4n29m5mxx040000gp/T/ipykernel_68069/2076384319.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  positions_df[losers] = -0.01
/var/folders/jv/chfp3b1d6yqbt4n29m5mxx040000gp/T/ipykernel_68069/2076384319.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  positions_df[losers] 

In [129]:
dyn_pos_ret = filtered_ret_df.mul(filtered_positions_df)
dyn_ret_df = dyn_pos_ret.dropna().sum(axis=1)

In [130]:
dyn_port_mean_ret = dyn_ret_df.mean() * SCALE
dyn_port_mean_ret

np.float64(0.0013787835706041416)

In [131]:
dyn_port_vol = dyn_ret_df.std() * np.sqrt(SCALE)
dyn_port_vol

np.float64(0.1010558889201944)

In [132]:
dyn_sharpe_ratio = dyn_port_mean_ret / dyn_port_vol
dyn_sharpe_ratio

np.float64(0.013643772622622626)

### 5. Market Exposure

Calculate a linear factor decomposition of your carry strategy (from part `2` or part `4`) against `SPY`.

Report the...
* alpha (annualized)
* beta
* r-squared

In [133]:
dyn_ret_df

date
2016-01-22   -0.000699
2016-01-29    0.000699
2016-02-05    0.027811
2016-02-12   -0.028922
2016-02-19    0.007107
                ...   
2024-11-29    0.004162
2024-12-06   -0.017658
2024-12-13    0.003819
2024-12-20   -0.001974
2024-12-27    0.000574
Length: 467, dtype: float64

In [134]:
benchmark_rets = ret_df = pd.read_excel('../data/spx_returns_weekly.xlsx', 'benchmark rets').set_index('date')
spy = benchmark_rets['SPY']

In [135]:
dyn_ret_df, spy = dyn_ret_df.align(spy, join='inner')
port_ret, spy = port_ret.align(spy, join='inner')

# Static

In [138]:
import statsmodels.api as sm

y = port_ret * SCALE
X = spy * SCALE
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Return   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     93.28
Date:                Wed, 15 Jan 2025   Prob (F-statistic):           3.10e-20
Time:                        20:59:12   Log-Likelihood:                -239.12
No. Observations:                 467   AIC:                             482.2
Df Residuals:                     465   BIC:                             490.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0020      0.019     -0.108      0.9

Report the...
* alpha (annualized) -0.0020
* beta -0.1477
* r-squared 0.167

# Dynamic

In [137]:
y = dyn_ret_df * SCALE
X = spy * SCALE
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.581
Date:                Wed, 15 Jan 2025   Prob (F-statistic):              0.209
Time:                        20:58:37   Log-Likelihood:                -513.56
No. Observations:                 467   AIC:                             1031.
Df Residuals:                     465   BIC:                             1039.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0069      0.034      0.204      0.8

Report the...
* alpha (annualized) 0.0069
* beta -0.0346
* r-squared 0.003

***